In [ ]:
import pandas as pd

In [ ]:
df= pd.read_excel("dataset_cosmetici.xlsx", dtype={'code': str})

In [ ]:
df['ingredients_text'].head(10)
df['code'].shape

(60915,)

In [ ]:
df['ingredients_tags'].head(2)

,ingredients_tags
0,NaN
1,"en:caratteristiche-prodotto,en:mentolo-e-con-o..."


In [ ]:
df['ingredients_analysis_tags'].head(2)

,ingredients_analysis_tags
0,NaN
1,"en:palm-oil-content-unknown,en:vegan-status-un..."


In [ ]:
df[['ingredients_analysis_tags','ingredients_tags','ingredients_text']].isnull().sum()

,0
ingredients_analysis_tags,43027
ingredients_tags,43667
ingredients_text,43655


In [ ]:
ingredients= df[['code','ingredients_text','ingredients_tags','ingredients_analysis_tags']].dropna()

In [ ]:
import pandas as pd
import re

df = pd.read_excel("dataset_cosmetici.xlsx", dtype={"code": str})

COLONNA_INGREDIENTI = "ingredients_text"

EXCLUDE = [
    "fragrance", "parfum",
    "limonene", "linalool", "citral", "geraniol",
    "phenoxyethanol", "ethylhexylglycerin",
    "benzyl benzoate"
]

STOPWORDS = [
    "caratteristiche", "prodotto", "base", "potenziato", "ricco",
    "con", "di", "a", "per", "e", "essenziale", "gel", "uso",
    "modo", "applicazione", "brasiliano", "molto"
]

TRANSLATION_MAP = {
    "acqua": "water",
    "olio di buriti": "buriti oil",
    "buriti": "buriti oil",
    "olio": "oil",
    "arnica": "arnica montana",
    "mentolo": "menthol",
    "timo": "thyme"
}

CANONICAL_INGREDIENTS = {
    # acqua
    "aqua": "water",
    "water": "water",
    "eau": "water",
    "agua": "water",

    # siliconi
    "silicone": "silicones",
    "silicones": "silicones",
    "siliconi": "silicones",

    # dimeticone
    "dimethicone": "dimethicone",
    "dimeticone": "dimethicone",

    # profumi
    "parfum": "fragrance",
    "fragrance": "fragrance",

    # mentolo
    "menthol": "menthol",
    "mentolo": "menthol",

    # oli
    "olio": "oil",
    "oil": "oil",

    # buriti
    "buriti oil": "buriti oil",
    "olio di buriti": "buriti oil",
    "buriti": "buriti oil",
}


def pulisci_ingrediente(text):
    text = re.sub(r"\(.*?\)", "", text)
    text = text.lower().strip()
    text = re.sub(r"[^a-zàèéìòù0-9\s\-]", "", text)
    return text

def normalize_ingredient(text):
    if not isinstance(text, str):
        return None

    words = [w for w in text.split() if w not in STOPWORDS]
    if not words:
        return None

    cleaned = " ".join(words)

    for it, en in TRANSLATION_MAP.items():
        if it in cleaned:
            cleaned = en

    if len(cleaned.split()) > 4:
        return None

    return cleaned.strip()

def canonicalize_ingredient(name):
    if not isinstance(name, str):
        return None

    name = name.strip().lower()

    if name in CANONICAL_INGREDIENTS:
        return CANONICAL_INGREDIENTS[name]

    for variant, canonical in CANONICAL_INGREDIENTS.items():
        if variant in name:
            return canonical

    return name

def is_functional(nome):
    for w in EXCLUDE:
        if w in nome:
            return 0
    return 1


rows = []

df_proc = df[["code", "product_name", COLONNA_INGREDIENTI]].dropna(
    subset=[COLONNA_INGREDIENTI]
)

for _, row in df_proc.iterrows():
    code = row["code"]
    product_name = row["product_name"]
    text = row[COLONNA_INGREDIENTI]

    for ingr in text.split(","):

        ingr = pulisci_ingrediente(ingr)
        ingr = normalize_ingredient(ingr)
        ingr = canonicalize_ingredient(ingr)

        if ingr:
            rows.append({
                "code": code,
                "product_name": product_name,
                "ingredient_name": ingr,
                "inci_name": ingr.title(),
                "is_functional": is_functional(ingr)
            })


ingredienti_normalizzati = (
    pd.DataFrame(rows)
    .drop_duplicates()
    .reset_index(drop=True)
)

ingredienti_normalizzati.insert(
    0,
    "ingredient_id",
    range(1, len(ingredienti_normalizzati) + 1)
)


ingredienti_normalizzati.to_excel(
    "ingredienti_normalizzati_FINAL.xlsx",
    index=False
)

✅ ingredienti_normalizzati_FINAL.xlsx creato
Righe totali: 273534


DOPO PRIMA RIUNIONE POST PRIMA PULIZIA.


In [ ]:
import pandas as pd

df = pd.read_excel("ingredienti_normalizzati_FINAL.xlsx",
    dtype={"code": str})

df = df[df["product_name"].notna()].copy()

if "ingredient_name" in df.columns:
    df = df.drop(columns=["ingredient_name"])

ingredient_map = {
    inci: idx + 1
    for idx, inci in enumerate(df["inci_name"].dropna().unique())
}

df["ingredient_id"] = df["inci_name"].map(ingredient_map)

df = df[
    [
        "code",
        "product_name",
        "ingredient_id",
        "inci_name",
        "is_functional"
    ]
]

df.to_excel("ingredienti_NORMALIZZATI_STRUTTURATI.xlsx", index=False)

